In [12]:
from matplotlib.pyplot import hot
import paramiko
from getpass import getpass

# Create the prompt for the password
password = getpass(prompt="Please Enter your SSH password: ")

# Hist Information
host = "192.168.1.229"
port = 22
username = "ubuntu"


try: 
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    # Open the SSH session
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication Failed.")

In [13]:
# Start SFTP Session
from threading import local

# File to Upload
file = "kraken"
# Start the SFTP Session
sftp = ssh.open_sftp()
# Copy the File
sftp.put(file, file)

# Make the file executable
command_chmod = "chmod +x ./kraken"
ssh.exec_command(command_chmod)

# Command to run the file
command_kraken = "sudo -S ./kraken --folder /usr/bin --folder /usr/sbin/ --folder /usr/local/bin --folder /sbin --folder /usr/local/sbin --folder /bin > navarro_kraken.txt"
# Run the Command 
stdin, stdout, stderr = ssh.exec_command(command_kraken)
# Pass user password to the sudo prompt
stdin.write(password + "\n")
# Hits the enter for the end of the command to finish
stdin.write("\n")
# The command above seems to be having issues when executing. I think it's getting terminated before completion
# This should make it wait till it finishes.
exit_status = stdout.channel.recv_exit_status()

# Pull the remote output file.
# Remote File
remote_file = "navarro_kraken.txt"
# Lets get the results
sftp.get(remote_file, "navarro_kraken.txt")

In [43]:
# Get matching processes out of the output file.
import re
pid_list = []
# Need to remove the cringe color code stuff.
no_ansi = []
# Regex to remove the ansi color settings
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
# Open the results file
with open("navarro_kraken.txt") as kraken_results:
    # Read the file
    lines = kraken_results.readlines()
    # loop through each line in the file
    for line in lines:
        # Remove the ansi color stuff
        line = ansi_escape.sub('', line)
        # Add the cleaned line to the no_ansi list
        no_ansi.append(line)
# Loop through the cleaned list
for line in no_ansi:
    # Search the item in the list for a PID string
    search = re.search(".*pid.*=.*", line)
    # If PID found then append it to the pid_list array.
    if search:
        # Pulls the specific PID out of the string
        pid = re.findall(r'pid=[0-9]{1,5}',line)
        if (len(pid) > 0):
            pid_list.append(pid[0])
print(pid_list)

['pid=825']


In [15]:
# Close the SSH Session
ssh.close()